# Parcial 1

In [ ]:
%%info

In [ ]:
# Configuración de Spark para habilitar el uso de un entorno virtual de Python
# Esta configuración asegura que PySpark utilice un entorno virtual específico para ejecutar el código.
# Se especifica el intérprete de Python, se habilita el uso de entornos virtuales y se define el tipo y la ruta del entorno virtual.

%%configure -f
{ "conf":{
          "spark.pyspark.python": "python",  # Especifica el intérprete de Python
          "spark.pyspark.virtualenv.enabled": "true",  # Habilita el uso de entornos virtuales
          "spark.pyspark.virtualenv.type":"native",  # Define el tipo de entorno virtual (nativo)
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"  # Ruta al ejecutable de virtualenv
         }
}

In [ ]:
# Importación de módulos y funciones necesarias para trabajar con PySpark y otras utilidades

# Spark
from pyspark.sql import SparkSession, DataFrame  # Para crear y manejar sesiones de Spark y DataFrames
from pyspark.sql.window import Window  # Para realizar operaciones de ventana en PySpark

# Tipos de datos
from pyspark.sql.types import (  # Para definir esquemas y tipos de datos en PySpark
    StringType, FloatType, IntegerType, DateType, StructType, StructField
)

# Funciones de PySpark
from pyspark.sql.functions import (  # Funciones comunes para manipulación de datos en PySpark
    col, lit, lower, trim, regexp_replace, udf
)

# Otros
import unicodedata  # Para normalización de texto (e.g., eliminar acentos)
from functools import reduce  # Para aplicar funciones acumulativas
import re  # Para trabajar con expresiones regulares

from pyspark.sql.functions import countDistinct  # Para contar valores distintos en una columna


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1746393038637_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Crear una sesión de Spark
# Esta configuración inicializa una sesión de Spark con el nombre "Profeco Parte A".
# La sesión de Spark es necesaria para ejecutar operaciones distribuidas en PySpark.

spark = SparkSession.builder \
    .appName("Profeco Parte A") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Configuración de variables y carga de datos desde S3

# Nombre del bucket en S3 (modificar según el nombre del usuario)
NAME = 'thmrudolf'  # CAMBIAR AQUÍ SU NOMBRE.
BUCKET = f"s3://itam-analytics-{NAME}"  # Ruta del bucket en S3
FOLDER = 'profeco'  # Carpeta dentro del bucket

# Tipo de catálogo a analizar
CATALOG_TYPE = 'basicos'

# Ruta de los archivos Parquet en S3
s3_path_parquet = f"{BUCKET}/{FOLDER}/parquet/"

# Cargar los datos desde los archivos Parquet en S3
df = spark.read.parquet(s3_path_parquet)

# Mostrar el esquema del DataFrame cargado
df.printSchema()

# Mostrar las primeras 5 filas de las columnas "anio", "catalogo" y "estado"
df.select("anio", "catalogo", "estado").show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- producto: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- precio: float (nullable = true)
 |-- fecha: date (nullable = true)
 |-- estado: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- anio: integer (nullable = true)

+----+--------+--------------+
|anio|catalogo|        estado|
+----+--------+--------------+
|2023| basicos|aguascalientes|
|2023| basicos|aguascalientes|
|2023| basicos|aguascalientes|
|2023| basicos|aguascalientes|
|2023| basicos|aguascalientes|
+----+--------+--------------+
only showing top 5 rows

## Parte A
En esta parte necesitarán levantar un cluster en AWS con Hadoop y Pyspark (Como lo hicimos en clase). Solo necesitan 1 cluster por equipo.

El nombre de tu cluster debe ser cluster_ + la mátricula (número de estudiante) más chica de los miembros del equipo. Por ejemplo: cluster_54903.
ETL con el Cluster.


Contesta las siguientes preguntas utilizando PySpark. Realiza el siguiente análisis (por año) y sobre todos los catálogos.

¿Cuántos catálogos diferentes tenemos?


In [ ]:
# Contar el número total de catálogos distintos en el DataFrame
# Utilizamos la función `distinct()` para obtener los valores únicos de la columna "catalogo"
# y luego aplicamos `count()` para contar cuántos valores únicos hay.

num_catalogos = df.select("catalogo").distinct().count()

# Imprimir el resultado
print(f"Total de catálogos distintos: {num_catalogos}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total de cat?logos distintos: 12

Respuesta: En total contamos con 12 catalógos.

¿Cuáles son los 20 catálogos con más observaciones? Guarda la salida de este query en tu bucket de S3, lo necesitaremos más adelante.


In [ ]:
# Obtener los 20 catálogos con más observaciones
# Agrupamos los datos por la columna "catalogo" y contamos el número de observaciones por catálogo.
# Luego, ordenamos los resultados en orden descendente por el conteo y seleccionamos los 20 primeros.

top_catalogos = df.groupBy("catalogo").count().orderBy(col("count").desc()).limit(20)

# Mostrar los resultados en la consola
top_catalogos.show()

# Guardar los resultados en S3
# Los resultados se guardan en formato Parquet en la ruta especificada en el bucket de S3.
s3_path_top_20 = f"{BUCKET}/{FOLDER}/top_20_catalogos/"
top_catalogos.write.mode("overwrite").parquet(s3_path_top_20)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+
|           catalogo|   count|
+-------------------+--------+
|            basicos|46965358|
|       medicamentos|19207468|
|  electrodomesticos| 7175494|
| frutas y legumbres| 5041527|
|   utiles escolares| 2936010|
|           mercados| 2238608|
|           juguetes| 1432183|
|              pacic| 1079162|
|pescados y mariscos|  569519|
|          navidenos|  236543|
|              tenis|   15768|
|        aeropuertos|     581|
+-------------------+--------+

**Respuesta:** En total contamos con 12 catalógos como resulto en la pregunta anterior, se muestra los 12 catalogos en la tabla arriba. El con mas observaciones es el *basico* el con menos *aeropuertos*. 

¿Tenemos datos de todos los estados del país? De no ser así, ¿cuáles faltan?


In [ ]:
# Obtener lista de estados únicos en los datos
# Se selecciona la columna "estado" del DataFrame, se eliminan duplicados con `distinct()`
# y se muestran los resultados (hasta 32 estados) en la consola.
estados_en_datos = df.select(col("estado")).distinct()
estados_en_datos.show(32)

# Guardar la lista de estados únicos en S3
# Los datos se guardan en formato Parquet en la ruta especificada en el bucket de S3.
s3_path_estados = f"{BUCKET}/{FOLDER}/estados/"
estados_en_datos.write.mode("overwrite").parquet(s3_path_estados)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|              estado|
+--------------------+
|          tamaulipas|
|           zacatecas|
|          nuevo leon|
|            campeche|
|     san luis potosi|
|            veracruz|
|             morelos|
|          guanajuato|
|              sonora|
|            tlaxcala|
|             nayarit|
|             sinaloa|
|              oaxaca|
|            guerrero|
|        quintana roo|
|           queretaro|
|    estado de mexico|
|              puebla|
|             durango|
|             jalisco|
|      aguascalientes|
|coahuila de zaragoza|
| baja california sur|
|              colima|
|             tabasco|
|           chihuahua|
|     baja california|
|    ciudad de mexico|
|             yucatan|
|             chiapas|
|             hidalgo|
| michoacan de ocampo|
+--------------------+

**Respuesta:** En un primer paso se analisa cuales son lo estados mencionados. Se ve en la lista arriba.

In [ ]:
# Lista de estados de México
# Esta lista contiene los nombres de los 32 estados de México, que se utilizarán para comparar con los datos disponibles.
estados_mexico = [
    "aguascalientes", "baja california", "baja california sur", "campeche", "coahuila de zaragoza",
    "colima", "chiapas", "chihuahua", "durango", "guanajuato", "guerrero", "hidalgo",
    "jalisco", "ciudad de mexico", "estado de mexico", "michoacan de ocampo", "morelos", "nayarit", "nuevo leon", "oaxaca",
    "puebla", "queretaro", "quintana roo", "san luis potosi", "sinaloa", "sonora",
    "tabasco", "tamaulipas", "tlaxcala", "veracruz", "yucatan", "zacatecas"
]

# Convertir PySpark DataFrame a lista para comparación
# Se extraen los estados únicos presentes en los datos y se convierten en una lista de Python.
estados_en_datos_lista = [row.estado for row in estados_en_datos.collect()]

# Encontrar estados faltantes
# Se calcula la diferencia entre la lista completa de estados de México y los estados presentes en los datos.
# Esto permite identificar los estados que no están representados en el DataFrame.
estados_faltantes = list(set(estados_mexico) - set(estados_en_datos_lista))

# Imprimir los estados faltantes
# Se muestra la lista de estados que no están presentes en los datos.
print(f"Estados faltantes en los datos: {estados_faltantes}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Estados faltantes en los datos: []

En un segundo paso, se define una lista con todos estados de México y se compara esta lista con la lista que se encontro en los datos. Resulta que todos los estados están mencionados en la base de datos.

¿Cuántas observaciones tenemos por estado?


In [ ]:
from pyspark.sql.functions import countDistinct

# Obtener el número de catálogos distintos por estado y año
# Agrupamos los datos por las columnas "estado" y "anio", y utilizamos la función `countDistinct`
# para contar el número de catálogos únicos en cada combinación de estado y año.
df_catalogos_por_estado_anio = df.groupBy("estado", "anio").agg(countDistinct("catalogo").alias("num_catalogos"))

# Mostrar resultados
# Se imprimen los resultados en la consola para verificar el número de catálogos distintos por estado y año.
df_catalogos_por_estado_anio.show()

# Guardar la salida en S3
# Los resultados se guardan en formato Parquet en la ruta especificada en el bucket de S3.
# Los datos se particionan por "estado" y "anio" para facilitar consultas posteriores.
s3_path_obs_por_estado = f"{BUCKET}/{FOLDER}/obs_por_estado_y_anio/"
df_catalogos_por_estado_anio.write.mode("overwrite") \
                            .partitionBy("estado", "anio") \
                            .option("compression", "snappy") \
                            .parquet(s3_path_obs_por_estado)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+-------------+
|             estado|anio|num_catalogos|
+-------------------+----+-------------+
|     aguascalientes|2021|           10|
|michoacan de ocampo|2020|            9|
|             oaxaca|2020|            9|
|            yucatan|2021|           10|
|             sonora|2023|           11|
|         nuevo leon|2023|           11|
|michoacan de ocampo|2023|           11|
|    baja california|2022|           11|
|     aguascalientes|2019|           11|
|           tlaxcala|2023|           11|
|          queretaro|2020|            9|
|          queretaro|2023|           11|
|baja california sur|2018|            9|
|            sinaloa|2021|            9|
|   ciudad de mexico|2019|           10|
|            hidalgo|2023|           11|
|            tabasco|2024|            9|
|           campeche|2023|           11|
|baja california sur|2021|           10|
|    baja california|2023|           11|
+-------------------+----+-------------+
only showing top

**Respuestas:** La tabla ilustra el nuemero de catalogos por estado y año. Se cuarda este informacion en un S3 as parquet para posible analisis futuros.

De cada estado obten: el número de catalogos diferentes por año, ¿ha aumentado el número de catálogos con el tiempo?
Utilizando Spark contesta las siguientes preguntas a partir del catálogo que le tocó a tu equipo. Recuerda trabajar en el archivo con los datos particionados de otra manera tus queries van a tardar mucho.

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, col

# Definir ventana por estado para comparar años anteriores
# Se utiliza una ventana particionada por "estado" y ordenada por "anio".
# Esto permite calcular métricas basadas en valores de años anteriores dentro de cada estado.
window_spec = Window.partitionBy("estado").orderBy("anio")

# Calcular la diferencia de catálogos con el año anterior
# Se utiliza la función `lag` para obtener el valor de "num_catalogos" del año anterior.
# Luego, se calcula la diferencia entre el número de catálogos del año actual y el año anterior.
catalogos_por_estado_anio = df_catalogos_por_estado_anio.withColumn(
    "diferencia_anual",
    col("num_catalogos") - lag("num_catalogos", 1).over(window_spec)
)

# Mostrar los primeros 10 resultados
# Se imprimen las primeras 10 filas del DataFrame resultante para verificar los cálculos.
catalogos_por_estado_anio.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----+-------------+----------------+
|         estado|anio|num_catalogos|diferencia_anual|
+---------------+----+-------------+----------------+
| aguascalientes|2018|            9|            NULL|
| aguascalientes|2019|           11|               2|
| aguascalientes|2020|            9|              -2|
| aguascalientes|2021|           10|               1|
| aguascalientes|2022|           11|               1|
| aguascalientes|2023|           11|               0|
| aguascalientes|2024|            9|              -2|
|baja california|2018|           10|            NULL|
|baja california|2019|           10|               0|
|baja california|2020|            9|              -1|
+---------------+----+-------------+----------------+
only showing top 10 rows

**Respuesta:** La tabla ilustra el numero de catalogos por año y estado. Hay poca variacion. Aguas Calientes por ejemplo tiene en promedio 10 catalogos. Hay que hacer una estadistica para cada estado (promedio, std) para ver que tanta variacion hay.

In [ ]:
# Guardar la salida en S3
# Los resultados se guardan en formato Parquet en la ruta especificada en el bucket de S3.
# Los datos se particionan por "estado" y "anio" para facilitar consultas posteriores.
s3_path_cat_dif_anio = f"{BUCKET}/{FOLDER}/catalogos_distintos/"
catalogos_por_estado_anio.write.mode("overwrite")\
                        .partitionBy("estado", "anio")\
                        .option("compression", "snappy") \
                        .parquet(s3_path_cat_dif_anio)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Guardar los resultados en S3 para su posterior análisis
# Los datos se guardan en formato Parquet en la ruta especificada en el bucket de S3.
# Se particionan por "estado" y "anio" para facilitar consultas posteriores y se utiliza compresión Snappy.

s3_path_estado_anio = f"{BUCKET}/{FOLDER}/catalogos_por_estado_anio/"
df_catalogos_por_estado_anio.write.mode("overwrite") \
                        .partitionBy("estado", "anio") \
                        .option("compression", "snappy") \
                        .parquet(s3_path_estado_anio)

# Imprimir la ruta donde se guardaron los resultados
print(f"Los resultados se han guardado en: {s3_path_estado_anio}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Los resultados se han guardado en: s3://itam-analytics-thmrudolf/profeco/catalogos_por_estado_anio/

In [ ]:
df_catalogos_por_estado_anio.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+-------------+
|             estado|anio|num_catalogos|
+-------------------+----+-------------+
|     aguascalientes|2021|           10|
|michoacan de ocampo|2020|            9|
|             oaxaca|2020|            9|
|            yucatan|2021|           10|
|             sonora|2023|           11|
|         nuevo leon|2023|           11|
|michoacan de ocampo|2023|           11|
|    baja california|2022|           11|
|     aguascalientes|2019|           11|
|           tlaxcala|2023|           11|
|          queretaro|2020|            9|
|          queretaro|2023|           11|
|baja california sur|2018|            9|
|            sinaloa|2021|            9|
|   ciudad de mexico|2019|           10|
|            hidalgo|2023|           11|
|            tabasco|2024|            9|
|           campeche|2023|           11|
|baja california sur|2021|           10|
|    baja california|2023|           11|
+-------------------+----+-------------+
only showing top

¿Cuańtas marcas diferentes tiene tu categoría?

In [ ]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------+--------------------+------+----------+--------------+--------------+--------+----+
|            producto|           marca|                tipo|precio|     fecha|        estado|        ciudad|catalogo|anio|
+--------------------+----------------+--------------------+------+----------+--------------+--------------+--------+----+
|              aceite|           1-2-3|aceites y grasas ...|  56.9|2023-04-17|aguascalientes|aguascalientes| basicos|2023|
|              aceite|          canoil|aceites y grasas ...|  49.9|2023-04-17|aguascalientes|aguascalientes| basicos|2023|
|              aceite|         capullo|aceites y grasas ...|  73.9|2023-04-17|aguascalientes|aguascalientes| basicos|2023|
|              aceite|          mazola|aceites y grasas ...|  71.5|2023-04-17|aguascalientes|aguascalientes| basicos|2023|
|              aceite|        nutrioli|aceites y grasas ...|  49.5|2023-04-17|aguascalientes|aguascalientes| basicos|2023|
|              a

In [ ]:
# Filtrar el DataFrame por el catálogo objetivo
# En este caso, se seleccionan únicamente las filas donde la columna "catalogo" tiene el valor "basicos".
catalogo_objetivo = "basicos"
df_filtered = df.filter(df["catalogo"] == catalogo_objetivo)

# Contar el número de marcas distintas en el catálogo objetivo
# Se utiliza la función `countDistinct` para contar las marcas únicas en la columna "marca".
# Además, se agrega una columna adicional "catalogo" para identificar el catálogo analizado.
num_marcas = df_filtered.select(countDistinct("marca").alias("num_marcas")).withColumn("catalogo", lit(catalogo_objetivo))

# Mostrar los resultados en la consola
# Se imprime el número de marcas distintas en el catálogo objetivo.
num_marcas.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+
|num_marcas|catalogo|
+----------+--------+
|       637| basicos|
+----------+--------+

**Respuesta:** El catalogo *basico* tiene 637 marcas.

¿Cuál es la marca con mayor precio? ¿En qué estado?


In [ ]:
# Obtener la fila con el precio máximo en el DataFrame
# Se ordena el DataFrame por la columna "precio" en orden descendente y se selecciona la primera fila.
# Esto permite identificar el producto con el precio más alto en los datos.

df_max_precio = df.orderBy(col("precio").desc()).limit(1)

# Mostrar los resultados
# Se imprime la fila con el precio máximo, incluyendo todas las columnas relevantes.
df_max_precio.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------+--------------------+--------+----------+------+------+-----------------+----+
| producto|  marca|                tipo|  precio|     fecha|estado|ciudad|         catalogo|anio|
+---------+-------+--------------------+--------+----------+------+------+-----------------+----+
|pantallas|samsung|aparatos electron...|114999.0|2018-03-16|puebla|puebla|electrodomesticos|2018|
+---------+-------+--------------------+--------+----------+------+------+-----------------+----+

**Respuesta:** La marca con mayor precio es una pantallas de samsung por 114999.0 MXN. Se venio en el estado de Puebla.

¿Cuál es la marca con menor precio en CDMX? (en aquel entonces Distrito Federal)


In [ ]:
# Obtener y mostrar los valores únicos de la columna "estado"
# Este código selecciona los valores únicos de la columna "estado" en el DataFrame `df`.
# Luego, utiliza `collect()` para convertir los resultados en una lista de filas.
# Finalmente, itera sobre cada fila y muestra el valor de la columna "estado".

unique_values = df.select("estado").distinct().collect()
for row in unique_values:
    print(row["estado"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tamaulipas
zacatecas
nuevo leon
campeche
san luis potosi
veracruz
morelos
guanajuato
sonora
tlaxcala
nayarit
sinaloa
oaxaca
guerrero
quintana roo
queretaro
estado de mexico
puebla
durango
jalisco
aguascalientes
coahuila de zaragoza
baja california sur
colima
tabasco
chihuahua
baja california
ciudad de mexico
yucatan
chiapas
hidalgo
michoacan de ocampo

In [ ]:
# Filtrar el DataFrame para obtener únicamente los datos de la Ciudad de México
# Se seleccionan las filas donde la columna "estado" tiene el valor "ciudad de mexico".
df_cdmx = df.filter(df['estado'] == 'ciudad de mexico')

# Obtener la fila con el menor precio en la Ciudad de México
# Se ordena el DataFrame por la columna "precio" en orden ascendente y se selecciona la primera fila.
df_cdmx_menor_precio_cdmx = df_cdmx.orderBy(col("precio").asc()).limit(1)

# Mostrar los resultados
# Se imprime la fila con el menor precio, incluyendo todas las columnas relevantes.
df_cdmx_menor_precio_cdmx.show()

# Guardar la salida en S3
# Los resultados se guardan en formato Parquet en la ruta especificada en el bucket de S3.
s3_path_marca_menor_prec = f"{BUCKET}/{FOLDER}/marca_menor_prec/"
df_cdmx_menor_precio_cdmx.write.mode("overwrite").parquet(s3_path_marca_menor_prec)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+----------------+------+----------+----------------+----------+----------------+----+
|        producto|marca|            tipo|precio|     fecha|          estado|    ciudad|        catalogo|anio|
+----------------+-----+----------------+------+----------+----------------+----------+----------------+----+
|pliegos de papel|  s/m|material escolar|  0.87|2018-08-21|ciudad de mexico|iztapalapa|utiles escolares|2018|
+----------------+-----+----------------+------+----------+----------------+----------+----------------+----+

**Respuesta:** La marca de menor precio en CDMX es s/m con el procuto pliegos de papel por 0.87 MXN.

¿Cuál es la marca con mayores observaciones?


In [ ]:
# Filtrar el DataFrame por el catálogo "basicos"
# Se seleccionan únicamente las filas donde la columna "catalogo" tiene el valor "basicos".
df_filtered_by_cat = df.filter(df['catalogo'] == 'basicos')

# Obtener el producto más frecuente en el catálogo "basicos"
# Agrupamos los datos por la columna "producto" y contamos el número de observaciones por producto.
# Luego, ordenamos los resultados en orden descendente por el conteo y seleccionamos el producto más frecuente.
df_producto_mas_frecuente = df_filtered_by_cat.groupBy("producto").count().orderBy(col("count").desc()).limit(1)

# Mostrar los resultados en la consola
# Se imprime el producto más frecuente en el catálogo "basicos".
df_producto_mas_frecuente.show()

# Guardar el producto más frecuente en S3
# Los resultados se guardan en formato Parquet en la ruta especificada en el bucket de S3.
s3_path_producto_mas_freq = f"{BUCKET}/{FOLDER}/producto_mas_freq/"
df_producto_mas_frecuente.write.mode("overwrite") \
                        .partitionBy("producto") \
                        .option("compression", "snappy") \
                        .parquet(s3_path_producto_mas_freq)

# Obtener la marca más frecuente en el catálogo "basicos"
# Agrupamos los datos por la columna "marca" y contamos el número de observaciones por marca.
# Luego, ordenamos los resultados en orden descendente por el conteo y seleccionamos la marca más frecuente.
df_marca_mas_frecuente = df_filtered_by_cat.groupBy("marca").count().orderBy(col("count").desc()).limit(1)

# Mostrar los resultados en la consola
# Se imprime la marca más frecuente en el catálogo "basicos".
df_marca_mas_frecuente.show()

# Guardar la marca más frecuente en S3
# Los resultados se guardan en formato Parquet en la ruta especificada en el bucket de S3.
s3_path_marca_mas_freq = f"{BUCKET}/{FOLDER}/marca_mas_freq/"
df_marca_mas_frecuente.write.mode("overwrite") \
                        .partitionBy("marca") \
                        .option("compression", "snappy") \
                        .parquet(s3_path_marca_mas_freq)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+
|producto|  count|
+--------+-------+
|refresco|2074163|
+--------+-------+

+-----+-------+
|marca|  count|
+-----+-------+
|  s/m|2762039|
+-----+-------+

**Respuesta:** La marca con mayores  observaciones es *s/m* con un total de 2762039 observaciones. Si lo vemos por producto, la mayor observaciones tiene *refrescos* con 2074163.

¿Cuáles son el top 5 de marcas con mayor precio en cada estado? ¿Son diferentes?


In [ ]:
from pyspark.sql.functions import col, row_number

# Calcular el precio máximo por marca y estado
# Agrupamos los datos por "estado", "marca" y "precio", y seleccionamos el precio como "max_precio".
# Esto nos permite identificar el precio máximo de cada marca en cada estado.
df_precio_por_marca_estado = df.groupBy("estado", "marca", "precio").agg(col("precio").alias("max_precio"))

# Aplicar ventana para obtener el top 5 de marcas con mayor precio por estado
# Definimos una ventana particionada por "estado" y ordenada por "max_precio" en orden descendente.
# Luego, asignamos un número de fila a cada registro dentro de cada partición utilizando `row_number()`.
# Filtramos los registros para quedarnos únicamente con las 5 marcas con mayor precio por estado.
window_spec = Window.partitionBy("estado").orderBy(col("max_precio").desc())
df_top5 = df_precio_por_marca_estado.withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= 5)

# Mostrar resultados
# Seleccionamos las columnas "estado", "marca" y "max_precio" del DataFrame resultante y mostramos los resultados.
df_top5.select("estado", "marca", "max_precio").show()

# Guardar la salida en S3
# Guardamos los resultados en formato Parquet en la ruta especificada en el bucket de S3.
# Los datos se particionan por "estado" y "marca" para facilitar consultas posteriores.
s3_path_5Top_marcas = f"{BUCKET}/{FOLDER}/5Top_marcas/"
df_top5.select("estado", "marca", "max_precio").write.mode("overwrite")\
                        .partitionBy("estado", "marca") \
                        .option("compression", "snappy")\
                        .parquet(s3_path_5Top_marcas)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------------+----------+
|             estado|       marca|max_precio|
+-------------------+------------+----------+
|     aguascalientes|sony. bravia|   95272.0|
|     aguascalientes|sony. bravia|   95236.0|
|     aguascalientes|sony. bravia|   95216.0|
|     aguascalientes|sony. bravia|   95116.0|
|     aguascalientes|sony. bravia|   95111.0|
|    baja california|     samsung|   68499.0|
|    baja california|     samsung|   64999.0|
|    baja california|     hisense|   55499.0|
|    baja california|sony. bravia|   53570.0|
|    baja california|     samsung|   51999.2|
|baja california sur|          lg|   69999.0|
|baja california sur|     samsung|   67149.0|
|baja california sur|          lg|   66099.0|
|baja california sur|     hisense|   55499.0|
|baja california sur|     samsung|   52856.0|
|           campeche|     samsung|  84125.56|
|           campeche|     samsung|   64442.0|
|           campeche|     samsung|   61999.0|
|           campeche|          lg|

**Respuesta:** Las top 5 de marcas con mayor precio en cada estado son diferentes (Sony, Samsung, Hisense, LG, etc. ) pero todos tienen en común que son electrodomesticos.

¿Cuáles son el top 5 de marcas con menor precio en CDMX? (en aquel entonces Distrito Federal)


In [ ]:
# Filtrar el DataFrame para obtener únicamente los datos de la Ciudad de México
# Se seleccionan las filas donde la columna "estado" tiene el valor "ciudad de mexico".
df_cdmx = df.filter(df['estado'] == 'ciudad de mexico')

# Definir una ventana para ordenar por precio ascendente
# La ventana no tiene partición, ya que se busca el top 5 de productos con menor precio en toda la Ciudad de México.
window_spec = Window.orderBy(col("precio").asc())

# Calcular el top 5 de productos con menor precio en la Ciudad de México
# Se utiliza la función `row_number()` para asignar un rango a cada fila basado en el precio ascendente.
# Luego, se filtran las filas para quedarse únicamente con las 5 primeras.
df_top5_cdmx = df_cdmx.withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= 5)

# Mostrar los resultados del top 5 de productos con menor precio
# Se seleccionan las columnas "producto", "estado" y "precio" para mostrar los resultados.
df_top5_cdmx.select("producto", "estado", "precio").show()

# Guardar los resultados del top 5 de productos con menor precio en S3
# Los datos se guardan en formato Parquet en la ruta especificada en el bucket de S3.
# Se particionan por "producto" para facilitar consultas posteriores.
s3_path_5Top_prod_menor_prec = f"{BUCKET}/{FOLDER}/5Top_prod_menor_prec/"
df_top5_cdmx.select("producto", "estado", "precio").write.mode("overwrite") \
                        .partitionBy("producto") \
                        .option("compression", "snappy") \
                        .parquet(s3_path_5Top_prod_menor_prec)

# Mostrar los resultados del top 5 de marcas con menor precio
# Se seleccionan las columnas "marca", "estado" y "precio" para mostrar los resultados.
df_top5_cdmx.select("marca", "estado", "precio").show()

# Guardar los resultados del top 5 de marcas con menor precio en S3
# Los datos se guardan en formato Parquet en la ruta especificada en el bucket de S3.
# Se particionan por "marca" para facilitar consultas posteriores.
s3_path_5Top_marcas_menor_prec = f"{BUCKET}/{FOLDER}/5Top_marcas_menor_prec/"
df_top5_cdmx.select("marca", "estado", "precio").write.mode("overwrite") \
                        .partitionBy("marca") \
                        .option("compression", "snappy") \
                        .parquet(s3_path_5Top_marcas_menor_prec)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+----------------+------+
|          producto|          estado|precio|
+------------------+----------------+------+
|  pliegos de papel|ciudad de mexico|  0.87|
|  pliegos de papel|ciudad de mexico|  0.97|
|  pliegos de papel|ciudad de mexico|   1.0|
|pan blanco bolillo|ciudad de mexico|   1.0|
|  pliegos de papel|ciudad de mexico|   1.0|
+------------------+----------------+------+

+-----+----------------+------+
|marca|          estado|precio|
+-----+----------------+------+
|  s/m|ciudad de mexico|  0.87|
|  s/m|ciudad de mexico|  0.97|
|  s/m|ciudad de mexico|   1.0|
|  s/m|ciudad de mexico|   1.0|
|  s/m|ciudad de mexico|   1.0|
+-----+----------------+------+

**Respuesta:** Las marcas con menor precio en CDMX son *s/m*, el tipo de producto son pliego papel y pan blanco.

¿Cuáles son el top 5 de marcas con mayores observaciones? ¿Se parecen a las de nivel por estado?


In [ ]:
# Obtener el conteo de observaciones por marca
# Agrupamos los datos por "marca", "estado" y "precio", y contamos el número de observaciones para cada combinación.
# Luego, ordenamos los resultados en orden descendente por el conteo y seleccionamos las 5 marcas con más observaciones.
df_top5_global = df.groupBy("marca", "estado", "precio").count().orderBy(col("count").desc()).limit(5)

# Mostrar los resultados en la consola
# Se imprimen las 5 marcas con más observaciones, incluyendo el estado y el precio asociado.
df_top5_global.show()

# Obtener el conteo de observaciones por producto
# Agrupamos los datos por "producto", "estado" y "precio", y contamos el número de observaciones para cada combinación.
# Luego, ordenamos los resultados en orden descendente por el conteo y seleccionamos los 5 productos con más observaciones.
df_top5_global_p = df.groupBy("producto", "estado", "precio").count().orderBy(col("count").desc()).limit(5)

# Mostrar los resultados en la consola
# Se imprimen los 5 productos con más observaciones, incluyendo el estado y el precio asociado.
df_top5_global_p.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------------+------+-----+
|marca|          estado|precio|count|
+-----+----------------+------+-----+
|  s/m|ciudad de mexico|  29.9|42847|
|  s/m|ciudad de mexico|  39.9|37643|
|  s/m|ciudad de mexico|  20.0|37178|
|  s/m|ciudad de mexico|  19.9|35388|
|  s/m|ciudad de mexico|  99.0|35107|
+-----+----------------+------+-----+

+--------+----------------+------+-----+
|producto|          estado|precio|count|
+--------+----------------+------+-----+
|refresco|ciudad de mexico|  22.0|29282|
|refresco|ciudad de mexico|  13.0|25663|
|refresco|ciudad de mexico|  25.0|22045|
|refresco|ciudad de mexico|  14.0|21041|
|refresco|ciudad de mexico|  23.0|17621|
+--------+----------------+------+-----+

**Respuesta:** Los top 5 de marcas con mayores observaciones *s/m*.

In [ ]:
# Guardar la salida en S3 (marca)
# Los resultados del top 5 de marcas con mayores observaciones se guardan en formato Parquet
# en la ruta especificada en el bucket de S3. Esto permite realizar análisis posteriores.
s3_path_5Top_marca_may_obs = f"{BUCKET}/{FOLDER}/5Top_marca_may_obs/"
df_top5_global.select("marca", "estado", "precio").write.mode("overwrite").parquet(s3_path_5Top_marca_may_obs)

# Guardar la salida en S3 (producto)
# Los resultados del top 5 de productos con mayores observaciones se guardan en formato Parquet
# en la ruta especificada en el bucket de S3. Esto facilita consultas y análisis futuros.
s3_path_5Top_prod_may_obs = f"{BUCKET}/{FOLDER}/5Top_prod_may_obs/"
df_top5_global_p.select("producto", "estado", "precio").write.mode("overwrite").parquet(s3_path_5Top_prod_may_obs)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Aplicar ventana para obtener el top 5 de marcas con mayores observaciones por estado
# Definimos una ventana particionada por "estado" y ordenada por el conteo de observaciones en orden descendente.
# Esto permite asignar un rango a cada marca dentro de cada estado basado en el número de observaciones.
window_spec = Window.partitionBy("estado").orderBy(col("count").desc())

# Calcular el top 5 de marcas con mayores observaciones por estado
# Agrupamos los datos por "estado" y "marca", contamos el número de observaciones por cada combinación,
# y aplicamos la ventana definida anteriormente para asignar un rango a cada marca.
# Luego, filtramos las filas para quedarnos únicamente con las 5 marcas con más observaciones por estado.
df_top5_estado = df.groupBy("estado", "marca").count() \
    .withColumn("rank", row_number().over(window_spec)) \
    .filter(col("rank") <= 5)

# Mostrar los resultados del top 5 de marcas con mayores observaciones por estado
df_top5_estado.show()

# Aplicar ventana para obtener el top 5 de productos con mayores observaciones por estado
# Similar al cálculo anterior, pero agrupamos los datos por "estado" y "producto" en lugar de "marca".
df_top5_estado_p = df.groupBy("estado", "producto").count() \
    .withColumn("rank", row_number().over(window_spec)) \
    .filter(col("rank") <= 5)

# Mostrar los resultados del top 5 de productos con mayores observaciones por estado
df_top5_estado_p.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------+------+----+
|             estado|     marca| count|rank|
+-------------------+----------+------+----+
|     aguascalientes|       s/m|597475|   1|
|     aguascalientes|       fud| 22276|   2|
|     aguascalientes|      mabe| 21556|   3|
|     aguascalientes|la costena| 21339|   4|
|     aguascalientes|     oster| 21129|   5|
|    baja california|       s/m|547912|   1|
|    baja california|      mabe| 22652|   2|
|    baja california|la costena| 22089|   3|
|    baja california|        lg| 19075|   4|
|    baja california|     oster| 16903|   5|
|baja california sur|       s/m|459203|   1|
|baja california sur|la costena| 23029|   2|
|baja california sur|      mabe| 19660|   3|
|baja california sur|     oster| 19280|   4|
|baja california sur|        lg| 19147|   5|
|           campeche|       s/m|581089|   1|
|           campeche|       fud| 25016|   2|
|           campeche|la costena| 24938|   3|
|           campeche|     oster| 23681|   4|
|         

In [ ]:
# Guardar la salida en S3 (marca)
s3_path_5Top_marca_may_obs_estado = f"{BUCKET}/{FOLDER}/5Top_marca_may_obs_estado/"
df_top5_global.select("marca", "estado", "precio").write.mode("overwrite")\
                    .partitionBy("estado", "marca")\
                    .option("compression", "snappy")\
                    .parquet(s3_path_5Top_marca_may_obs_estado)

# Guardar la salida en S3 (producto)
s3_path_5Top_prod_may_obs_estado = f"{BUCKET}/{FOLDER}/5Top_prod_may_obs_estado/"
df_top5_global_p.select("producto", "estado", "precio").write.mode("overwrite")\
                    .partitionBy("estado", "producto")\
                    .option("compression", "snappy")\
                    .parquet(s3_path_5Top_prod_may_obs_estado)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 ¿Se parecen a las de nivel por estado?

In [ ]:
from pyspark.sql.functions import collect_list

# Obtener lista de marcas en el top global
marcas_top_global = [row["marca"] for row in df_top5_global.collect()]

# Agrupar marcas del top 5 por estado y compararlas con el top global
df_comparacion = df_top5_estado.groupBy("estado").agg(collect_list("marca").alias("top_5_marcas"))

df_comparacion.show(truncate=False)

# Guardar la salida en S3
s3_path_5Top_marca_comparacion_estado = f"{BUCKET}/{FOLDER}/5Top_marca_comparacion_estado/"
df_comparacion.write.mode("overwrite")\
                    .partitionBy("estado")\
                    .option("compression", "snappy")\
                    .parquet(s3_path_5Top_marca_comparacion_estado)


# Obtener lista de marcas en el top global
producto_top_global = [row["producto"] for row in df_top5_global_p.collect()]

# Agrupar marcas del top 5 por estado y compararlas con el top global
df_comparacion_p = df_top5_estado_p.groupBy("estado").agg(collect_list("producto").alias("top_5_productos"))

df_comparacion_p.show(truncate=False)

# Guardar la salida en S3
s3_path_5Top_prod_comparacion_estado = f"{BUCKET}/{FOLDER}/5Top_prod_comparacion_estado/"
df_comparacion_p.write.mode("overwrite")\
                    .partitionBy("estado")\
                    .option("compression", "snappy")\
                    .parquet(s3_path_5Top_prod_comparacion_estado)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------------------------------------+
|estado              |top_5_marcas                                  |
+--------------------+----------------------------------------------+
|aguascalientes      |[s/m, fud, mabe, la costena, oster]           |
|baja california     |[s/m, mabe, la costena, lg, oster]            |
|baja california sur |[s/m, la costena, mabe, oster, lg]            |
|campeche            |[s/m, fud, la costena, oster, mabe]           |
|chiapas             |[s/m, la costena, mabe, oster, bimbo]         |
|chihuahua           |[s/m, la costena, mabe, lg, bimbo]            |
|ciudad de mexico    |[s/m, la costena, fud, bimbo, lala]           |
|coahuila de zaragoza|[s/m, la costena, mabe, fud, lg]              |
|colima              |[s/m, la costena, lg, fud, mabe]              |
|durango             |[s/m, oster, lg, la costena, mabe]            |
|estado de mexico    |[s/m, la costena, fud, bimbo, lala]           |
|guanajuato         

¿Ha dejado de existir alguna marca durante los años que tienes? ¿Cuál? ¿Cuándo desapareció?


In [ ]:

# Obtener marcas únicas por año
df_marcas_por_anio = df.select("anio", "marca").distinct()

# Obtener todas las marcas que existieron alguna vez
marcas_existentes = df_marcas_por_anio.select("marca").distinct()

# Obtener la última aparición de cada marca
df_ultima_aparicion = df_marcas_por_anio.groupBy("marca").agg({"anio": "max"}).withColumnRenamed("max(anio)", "ultimo_anio")

# Comparar con las marcas actuales (último año presente en los datos)
ultimo_anio = df.select("anio").distinct().orderBy("anio", ascending=False).limit(1).collect()[0]["anio"]
df_marcas_desaparecidas = df_ultima_aparicion.filter(df_ultima_aparicion["ultimo_anio"] < ultimo_anio)

# Mostrar resultados
df_marcas_desaparecidas.show()

# Guardar la salida en S3
s3_path_marcas_deaparecidas = f"{BUCKET}/{FOLDER}/marcas_deaparecidas/"
df_marcas_desaparecidas.write.mode("overwrite")\
                    .partitionBy("marca")\
                    .option("compression", "snappy")\
                    .parquet(s3_path_marcas_deaparecidas)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+
|               marca|ultimo_anio|
+--------------------+-----------+
|             kleenex|       2023|
|osram. dulux el m...|       2023|
|kleenex. cottonel...|       2022|
|             lyncott|       2023|
|osram. dulux valu...|       2023|
|        alpura. kids|       2022|
|colgate. luminous...|       2021|
|    alpura. vaquitas|       2022|
|  petalo. ultra care|       2022|
|         mundet lift|       2021|
|quality day. led ...|       2023|
| savile. hidratacion|       2021|
|            frutimax|       2023|
|l oreal. paris. c...|       2021|
|colgate. luminous...|       2023|
|    oral-b. 3d white|       2021|
|gran cosecha. pre...|       2023|
|oral-b. 3d white....|       2023|
|soriana. espiral....|       2021|
|svelty. con colag...|       2023|
+--------------------+-----------+
only showing top 20 rows

Genera una gráfica de serie de tiempo por estado para la marca con mayor precio -en todos los años-, donde el eje equis es el año y el eje ye es el precio máximo.
Nota: Recuerden descargar del cluster su análisis en Jupyter, de otra manera se borrará.

Hint: Guarda tus consultas en archivos que puedas guardar en S3 y luego leer desde Pandas o RStudio, para hacer tus gráficas o cuadros compartivos.

In [ ]:
spark._sc.install_pypi_package("pandas")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Not uninstalling python-dateutil at /usr/lib/python3.9/site-packages, outside environment /mnt/yarn/usercache/livy/appcache/application_1746393038637_0002/container_1746393038637_0002_01_000001/tmp/spark-8cc307e1-a7f7-4fd7-aea3-111a06d1dce8
    Can't uninstall 'python-dateutil'. No files were found to uninstall.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.23.11 requires python-dateutil<=2.9.0,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.

In [ ]:
spark._sc.install_pypi_package("matplotlib", "https://pypi.org/simple")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:


# Inicializar Spark Session
spark = SparkSession.builder.appName("Time Series Analysis").getOrCreate()

# Cargar datos desde S3
df = spark.read.parquet("tbd")

# Obtener la marca con el mayor precio en todos los años
df_max_precio_marca = df.groupBy("estado", "anio", "marca").agg(col("precio").alias("max_precio")) \
    .orderBy(col("max_precio").desc()).dropDuplicates(["anio", "estado"])

# Convertir a Pandas para graficar
df_pandas = df_max_precio_marca.toPandas()

# Crear gráfica de serie de tiempo
plt.figure(figsize=(12, 6))
for estado in df_pandas["estado"].unique():
    data_estado = df_pandas[df_pandas["estado"] == estado]
    plt.plot(data_estado["anio"], data_estado["max_precio"], marker="o", label=estado)

plt.xlabel("Año")
plt.ylabel("Precio Máximo")
plt.title("Evolución del Precio Máximo por Estado (Marca Más Cara)")
plt.legend()
plt.grid(True)
plt.show()
%matplot plt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
[PATH_NOT_FOUND] Path does not exist: hdfs://ip-172-31-5-244.ec2.internal:8020/user/livy/tbd.
Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1746393038637_0002/container_1746393038637_0002_01_000001/pyspark.zip/pyspark/sql/readwriter.py", line 544, in parquet
    return self._df(self._jreader.parquet(_to_seq(self._spark._sc, paths)))
  File "/mnt/yarn/usercache/livy/appcache/application_1746393038637_0002/container_1746393038637_0002_01_000001/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/mnt/yarn/usercache/livy/appcache/application_1746393038637_0002/container_1746393038637_0002_01_000001/pyspark.zip/pyspark/errors/exceptions/captured.py", line 185, in deco
    raise converted from None
pyspark.errors.exceptions.captured.AnalysisException: [PATH_NOT_FOUND] Path does not exist: hdfs://ip-172-31-5-244.ec2.internal:8020/user/livy/tbd.



In [ ]:
s3_path_df_complete = f"{BUCKET}/{FOLDER}/df_complete/"
df.write.mode("overwrite")\
                    .partitionBy("Estado", "anio", "catalogo")\
                    .option("compression", "snappy")\
                    .parquet(s3_path_df_complete )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_basicos = df.filter(df['catalogo']=='basicos')
s3_path_df_complete_basicos = f"{BUCKET}/{FOLDER}/df_complete_basicos/"
df_basicos.write.mode("overwrite")\
                    .partitionBy("Estado", "anio",)\
                    .option("compression", "snappy")\
                    .parquet(s3_path_df_complete_basicos)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…